### Load necessary libraries

In [53]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from statsmodels.stats.outliers_influence import variance_inflation_factor

### Load data

In [54]:
df = pd.read_csv('music_copyright_rate.csv', encoding='utf-8')


In [55]:
df.loc[0]
df_refined= df.drop(df.index[[6,7,25,26,27,28,29,30,31,32,33,22]])
df_refined

,name,sales,bidirectional,place_unlimit,time_unlimit,repeat_consm,purpose,rate,Unnamed: 8
0,주요 3사,3279613,0,0,0,0,0,0.0120,NaN
1,지역방송,492188,0,0,0,0,0,0.0100,NaN
2,교육방송,191955,0,0,0,0,0,0.0035,NaN
3,"종교방송, 지역라디오",87963,0,0,0,0,0,0.0120,NaN
4,교통방송,10884,0,0,0,0,0,0.0135,NaN
5,극동방송,2486,0,0,0,0,0,0.0070,NaN
8,국악방송,7267,0,0,0,0,1,0.0070,NaN
9,YTN라디오,6014,0,0,0,0,0,0.0060,NaN
10,홈쇼핑 PP,4719138,0,0,0,0,0,0.0250,NaN
11,음악 PP,4803,0,0,0,0,1,0.0400,NaN


In [56]:
df_refined.describe()

,sales,bidirectional,place_unlimit,time_unlimit,repeat_consm,purpose,rate
count,2.200000e+01,22.000000,22.000000,22.000000,22.000000,22.000000,22.000000
mean,8.276637e+05,0.181818,0.136364,0.090909,0.181818,0.227273,0.020295
std,1.498878e+06,0.394771,0.351250,0.294245,0.394771,0.428932,0.029437
min,1.814000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.003500
25%,8.171250e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.006250
50%,1.724850e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000
75%,5.024548e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.013125
max,4.894536e+06,1.000000,1.000000,1.000000,1.000000,1.000000,0.110000


#### Unrestriced Model

In [65]:
df_refined['log_sales'] = np.log(df_refined['sales'])
y = df_refined['rate']
X = df_refined[[
    'log_sales',
    'bidirectional', 
    'place_unlimit', 
    'time_unlimit', 
    'repeat_consm',
    'purpose'
    ]]
X_cons = sm.add_constant(X)

#### Restricted Model

In [62]:
df_refined['log_sales'] = np.log(df_refined['sales'])
y = df_refined['rate']
X = df_refined[[
    'log_sales',
    # 'bidirectional', 
    # 'place_unlimit', 
    'time_unlimit', 
    # 'repeat_consm',
    'purpose'
    ]]
X_cons = sm.add_constant(X)

### Check Multicollinearity between independent variables

In [66]:
# Initialize an empty DataFrame for VIF data
vif_data = pd.DataFrame()

# Add feature names from X
vif_data["feature"] = X.columns

# Calculate VIF for each feature in X
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# Display the VIF data
print(vif_data)

         feature       VIF
0      log_sales  1.472724
1  bidirectional       inf
2  place_unlimit  3.104732
3   time_unlimit  5.069667
4   repeat_consm       inf
5        purpose  1.832918


/Users/choeseung-u/programming/econometrics/econometrics/lib/python3.11/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


### Fit OLS

In [67]:
model_ols = sm.OLS(y,X_cons)
# ols_result = model_ols.fit(cov_type='HC3') #Heteroskedasticity Robust Standard Error
ols_result = model_ols.fit() # normal standard error
print(ols_result.summary())

                            OLS Regression Results                            
Dep. Variable:                   rate   R-squared:                       0.949
Model:                            OLS   Adj. R-squared:                  0.933
Method:                 Least Squares   F-statistic:                     59.34
Date:                Sat, 09 Dec 2023   Prob (F-statistic):           9.26e-10
Time:                        15:17:00   Log-Likelihood:                 79.556
No. Observations:                  22   AIC:                            -147.1
Df Residuals:                      16   BIC:                            -140.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0002      0.011      0.015

#### Check heteroskedasticity

In [52]:
test_statistic_br, p_value_br, _, _ = het_breuschpagan(ols_result.resid,ols_result.model.exog)
print("Breusch-Pagan Test Statistic:", test_statistic_br)
print("P-Value:", p_value_br)

test_statistic_wh, p_value_wh, _, _ = het_white(ols_result.resid, ols_result.model.exog)
print("White Test Statistic:", test_statistic_wh)
print("P-Value:", p_value_wh)

Breusch-Pagan Test Statistic: 10.825205795039835
P-Value: 0.012709595305077453
White Test Statistic: 17.36175925488714
P-Value: 0.008041757293661309
